# Solving the forward problem using PiNNs

## Load Libraries

In [1]:
import deepxde as dde
from deepxde.backend import tf
import numpy as np
from deepxde.callbacks import EarlyStopping
import matplotlib.pyplot as plt

from scipy.io import loadmat
m_path = '../data/drop.mat'
records = loadmat(m_path, squeeze_me=True, mat_dtype=True)['a']

Using TensorFlow 2 backend.

Instructions for updating:
non-resource variables are not supported in the long term


## Pendant Drop Equations

We use the young laplace formula and solve for the curvatures

$
\begin{align}
& \sigma(\kappa_s+\kappa_\psi) = p_L - \Delta \rho g z \\
\kappa_s = \frac{\text{d}\psi}{\text{d}{s}} = -\frac{\frac{d^2r}{ds^2}}{\sqrt{1-(dr/ds)^2}}\hspace{1em} & \kappa_{\phi} = \frac{\text{sin}(\psi)}{r}  = \frac{1}{r}\frac{dz}{ds}\hspace{1em} & \psi = \text{tan}^{-1} \left( \frac{dz}{dr} \right) \\
\end{align}
$

With the following boundary conditions
$\begin{align} @s=0, z=0, r=0, \psi = 0  ;  @s=L, r=r_{needle};   \end{align} $



## Constants

In [2]:
sigma = 72.0;        # surface tension [mN/m]
grav = 1.0;     # gravitational acceleration [mm/s^2]
rneedle = 1.0;       # radius of the needle [mm]
deltarho = 1.0;   # density difference [10^6 kg/m^3]
p_l = 70.2799663375; # laplace pressure
s = 5.1947
eps=1e-8


In [4]:
# convention, r = 0 , z=1, psi=2
def pde(s,y):
    
    r = y[:,0:1]
    z = y[:,1:2]
    psi = y[:,2:3]
    
    dr_ds = dde.grad.jacobian(r, s)
    dz_ds = dde.grad.jacobian(z, s)
    dpsi_ds = dde.grad.jacobian(psi, s)

    eqn_1 = dr_ds - tf.math.cos(psi)
    eqn_2 = dz_ds - tf.math.sin(psi)
    eqn_3 = - p_l + z + sigma* (dpsi_ds + tf.math.divide_no_nan(tf.math.sin(psi),r)) 
    
    
    return  [eqn_1, eqn_2, eqn_3]


geom = dde.geometry.Interval(0, s)

def bot_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], 0)

def top_boundary(x, on_boundary):
    return on_boundary and np.isclose(x[0], s)

bot_bc_r = dde.DirichletBC(geom, lambda x: 0, bot_boundary, component = 0)  # r(0) = 0
bot_bc_z = dde.DirichletBC(geom, lambda x: 0, top_boundary, component = 1) # z(0) = 0
top_bc_r = dde.DirichletBC(geom, lambda x: 1, top_boundary, component=0) # r(L) = r_needle
apex_bc = dde.DirichletBC(geom, lambda x: 0, bot_boundary, component=2) #psi(0) = 0

data = dde.data.PDE(
    geom,
    pde,
    [bot_bc_r,bot_bc_z,top_bc_r,apex_bc],
    num_domain=1000,
    num_boundary=50,
    num_test=3000
)

layer_size = [1] + [20] * 5 + [3]
activation = "tanh"
initializer = "Glorot uniform"
net = dde.maps.FNN(layer_size, activation, initializer)
# net.apply_output_transform(lambda x, y: y*2)

model = dde.Model(data, net)

model.compile("adam", lr=0.001)
model.restore('../models/review.ckpt-100000')
early_stopping = EarlyStopping(min_delta=1e-8, patience=40000)
losshistory, train_state =  model.train(epochs=100000, display_every=10000, callbacks=[early_stopping])

Compiling model...
Building feed-forward neural network...
'build' took 0.073887 s

'compile' took 1.212731 s

INFO:tensorflow:Restoring parameters from ../models/review.ckpt-100000
Initializing variables...
Training model...

0         [1.04e+00, 5.84e-02, 2.38e+04, 0.00e+00, 3.72e-01, 1.28e+00, 0.00e+00]    [1.04e+00, 5.79e-02, 2.48e+04, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00]    []  


KeyboardInterrupt: 

In [ ]:
predicted.T.shape

In [ ]:
# X_train, y_train, X_test, y_test, best_y, best_ystd = train_state.packed_data()
data = geom.uniform_points(100)
predicted = model.predict(data)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,6))
ax[0].plot(predicted.T[0] ,predicted.T[1], label='PINN Solution' )
ax[0].set_xlabel('r')
ax[0].set_ylabel('z', rotation=0)

ax[0].plot(records.T[0] ,records.T[1], label='Ground Truth' )
ax[0].set_xlabel('r')
ax[0].set_ylabel('z', rotation=0)
ax[0].legend()


loss_train = np.sum(losshistory.loss_train, axis=1)
loss_test = np.sum(losshistory.loss_test, axis=1)

ax[1].semilogy(losshistory.steps, loss_train, label="Train loss")
ax[1].semilogy(losshistory.steps, loss_test, label="Test loss")
for i in range(len(losshistory.metrics_test[0])):
    ax[1].semilogy(
        losshistory.steps,
        np.array(losshistory.metrics_test)[:, i],
        label="Test metric",
    )
ax[1].set_xlabel("# Steps")
ax[1].set_ylabel("Losses")
ax[1].legend()

plt.tight_layout()

In [ ]:
predicted[2]